In [1]:
from nn.nn import NN
import gymnasium as gym
from agent.agent import AgentBuilder, ActionContext
from experience.experience import ExperienceBuffer
import torch
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
ENV_NAME = 'FrozenLake-v1'
EPSILON = 1
EPSILON_DECAY = 1000
EPSILON_DECAY_VALUE = 0 
POLICY = 'GREEDY'
ACTION_METHOD = 'NEURAL_NETWORK'
EXPERIENCE_BUFFER = 10
SAMPLE_SIZE = 10
LEARNING_RATE = 0.01
GAMMA = 0.1
HIDDEN_DIMENSION = (16, 16)
A_FUNCTION = torch.nn.ReLU()
LOSS_F=torch.nn.MSELoss()
OPTIMIZER=torch.optim.Adam

Create environment to play with:

In [3]:
    
env = gym.make(ENV_NAME, desc=None, map_name="4x4", is_slippery=False,
                render_mode="rgb_array")
starting_state, _ = env.reset()

In [4]:
def print_frame(env_, number):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
    ax.imshow(env_.render())
    ax.axis("off")
    ax.set_title(number)
    fig.show()

Create prediction/on-line neural network.

In [5]:

nn = NN(
    nb_of_state_variables = 1,
    nb_of_actions = env.action_space.n,
    activation_functions= A_FUNCTION,
    hidden_dims=HIDDEN_DIMENSION
    )

Some theory:

Assumption for supervised learning with gradient descent:
1) data must by independent
2) identically distributed
3) target needs to be stationary
However for reinforcement learning these assumption are not kept.

Three classic optimization technics in ML:
1) Batch gradient descent -> taking all samples at once. Not practical -> to large data sets
2) Mini batch gradient descent -> taking small portion of samples
3) Stochastic gradient descent -> taking one sample per step. High variance per step of optimization. Less stable then mini-batch gradient descent
The bigger the sample space, the lower the variance of the steps. To large sample to slow process of learning. In order to use mini-batch gradient descent experience is gathered.

Problems with classical RL methods:

* Problem 1 - Collected samples comes from the same trajectory and policy. Experience gathered in state t+1 depends on the state t. So experience will looks like (t -> t+1 -> t+2 -> t+3). In each time information about variables of a current state, variables of a previous state and reward are kept. When target network is updated and then samples are collected a mini-batch comes from older policy. What's more batches are correlated with each other. It violate assumptions that data are independent (correlation between batches) and identically distributed (policy is changing so the samples distribution are changing)


* Problem 2 - Weights are optimized for all states at once. When using the same target network and optimization network it creates non-stationary target. This means that the values estimated by a target network maybe outdated. Moreover target shifts its values when the network weights are updated.

Value_estimated_by_neural_network_for_current_state = Reward + gamma * max(Value_estimated_by_neural_network_for_next_state)

Solving issues with IID:
Solution to the Problem 1: Using batches -> Collecting data with use of the same neural network. Improvements to that solution could be use of replay buffer. Having ExperienceBuffer from which samples are taken much higher than batches use for learning. Old diverse experience can halp learning. Samples are information about state and action and consequence of this action (variables describing next state and reward).

Solution with a framework: \
size_of_buffer >> size_of_sample

eg. \
    exp = ExperienceBuffer(size_of_buffer=1000)
    samples = exp.get_samples(100)

Important: Neural Networks are used to estimate consequences of a transition. Loss function is used to evaluate how consequences were estimated.

In [6]:
exp = ExperienceBuffer(
    size_of_buffer=EXPERIENCE_BUFFER, 
    sampling_method='SIMPLE_RANDOM'
    )

Solution to the Problem 2:  Using different target network -> it tends to solve problems with non-stationary of a target. One network is used to evaluate value of a target (Value_estimated_by_neural_network_for_next_state) and another is used to optimize its weights to better estimate consequences of a transitions between two states after taking action. After several iteration weights of both neural networks are synchronized. 

Remark: This are not really a different network. The architecture is the same only weights are different.

In order to use different target network synchronization attribute must be set:

In [7]:

action_context = ActionContext(
    actions_number=env.action_space.n,
    action_methods=ACTION_METHOD,
    epsilon = EPSILON,
    nn = nn,
    policy= POLICY,
    epsilon_decay=EPSILON_DECAY,
    epsilon_decay_value=EPSILON_DECAY_VALUE
)

agent = AgentBuilder(
    action_context,
    env=env,
    gamma=GAMMA,
    loss_function=LOSS_F,
    optimizer=OPTIMIZER,
    learning_rate=LEARNING_RATE,
    synchronization = SAMPLE_SIZE # after how many step synchronization will be made
    ).build_agent()

agent.trigger_decay = False

In [8]:
counter = 0
episode_steps = 0

episode_steps_all = []
while counter <= 100000:
    
    if counter % 10000 == 0:
        print(f'Step nb: {counter}')
    
    # gathering experience
    starting_state = torch.tensor([starting_state], dtype=torch.float)
    starting_state.unsqueeze(0)
    
    action = agent.compute_action(starting_state)
    exp_s = agent.make_step(action, starting_state[0].item())
    #print(f'state: {exp_s.state}, previous_state: {exp_s.previous_state}, action: {exp_s.action}')
    #print_frame(agent.env, counter)
    exp.append(exp_s)
    starting_state = exp_s.state
    if exp_s.terminated:
        starting_state, _ = agent.env.reset()
        if exp_s.reward == 1:
            agent.trigger_decay = True
            print(f'Episode ended after: {episode_steps}, Reward = {exp_s.reward}')
            
        episode_steps_all.append(episode_steps)
        episode_steps = 0
        
    
    # train agent
    if len(exp) > SAMPLE_SIZE:
        samples = exp.get_samples(SAMPLE_SIZE)
        agent.train_agent(samples)
        
    # synchronization
    agent.synchronize(counter)
    
    counter +=1
    episode_steps +=1

Step nb: 0
Episode ended after: 7, Reward = 1.0
Step nb: 10000
Step nb: 20000
Step nb: 30000
Step nb: 40000
Step nb: 50000
Step nb: 60000
Step nb: 70000
